In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from DF_DeLt import DeepFace
from DF_DeLt.commons import distance as dst
from DF_DeLt.detectors import FaceDetector
import time
import cv2
from glob import glob as gl
import numpy as np
from tqdm import trange
from matplotlib import pyplot as plt
import threading

models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
model_name = "Facenet"
recognizor = DeepFace.build_model(model_name)

In [ ]:
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs, convert_output

num_classes = 1
tfrecord = None
weights = 'face-yolov3-tiny.tf'
classes = './yolov3_tf2/face.names'
size = 416
yolo = YoloV3Tiny(classes=num_classes)
yolo.load_weights(weights).expect_partial()

def face_detection(img_raw):
    img_orin = img_raw.copy()
    img_raw = tf.convert_to_tensor(img_raw, dtype=tf.uint8)
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, size)
    boxes, scores, classes, nums = yolo(img)
    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    output = convert_output(img, (boxes, scores, classes, nums))
    outputs = []
    for out in output:
        top, right, bottom, left = out
        top = 0 if top <0 else top
        right = 0 if right <0 else right
        bottom = 0 if bottom <0 else bottom
        left = 0 if left <0 else left
        face = img_orin[top:bottom, left:right]
        outputs.append([face, [left, top, right, bottom]])
    return outputs

In [ ]:
detector_names = ['opencv','ssd','mtcnn', 'retinaface']
detector_name = 'ssd'
detector = FaceDetector.build_model(detector_name)

def face_detection(img_raw, align=False, single=True):
    faces = FaceDetector.detect_face(detector, detector_name, img_raw, align , single)
    return faces

In [ ]:
def distance(e1, e2):
    distance = dst.findEuclideanDistance(dst.l2_normalize(e1), dst.l2_normalize(e2))
    return distance

def show(img):
    plt.imshow(img[:, :, ::-1])
    plt.show()

In [ ]:
from tqdm import tnrange
def load_user(path):
    images = gl(path+"/*.jpg")
    info_file = path+"/info.txt"
    f = open(info_file, "r")
    content = f.readlines()
    info = {"uid": content[0][:-1],
             "name": content[1][:-1],
             "age": content[2][:-1],
             "permission": content[3][:-1],
             "more": content[4][:-1]
            }
    encoded_faces = []
    for image in images:
        img = cv2.imread(image)
        try:
            face = face_detection(img)[0][0]
            encoded_face = DeepFace.represent(face, model = recognizor, fast=True)
            encoded_faces.append(encoded_face)
        except:
            pass
        
    return {"info":info, "vectors": encoded_faces}

def load_data():
    paths = gl("users/*")
    data = {}
    for i in tnrange(len(paths)):
        path = paths[i]
        user = load_user(path)
        data[user.get("info").get("uid")] = user
    return data

def distances_compute(encoded_face, user):
    vectors = user['vectors']
    distances = []
    for vector in vectors:
        distances.append(distance(vector, encoded_face))
    return np.array(distances)

def user_verify(face, threshold = 0.3):
    min_distance = 2
    found_key = None
    user_matched = None
    h, w = face.shape[:2]
    encoded_face = DeepFace.represent(face, model = recognizor, fast=True)
    for key in keys:
        distances = distances_compute(encoded_face, data[key])
        mean_dis, min_dis = np.mean(distances), np.min(distances)
        if min_dis < min_distance:
            found_key = key
            min_distance = min_dis
            user_matched = data[found_key]["info"]
    if min_distance<threshold:
        return min_distance, user_matched
    else:
        return min_distance, None

In [ ]:
data = load_data()
keys = list(data.keys())

In [ ]:
threshold = 0.9
test_image = cv2.imread("3.jpg")
show(test_image)
import time
tik = time.time()
# faces detection
faces = face_detection(test_image)
tok = time.time()
print("detect:", tok -tik)
# face = cv2.imread("2.jpg")
for face in faces:
    show(face[0])
    tik = time.time()
    # face verify 
    min_distance, user_matched = user_verify(face[0], threshold = threshold)
    tok = time.time()
    print("recognition", tok -tik)
    print(min_distance)
    print(user_matched)

In [ ]:
type_of_trackers = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 
                     'MOSSE', 'CSRT']                                          
def generate_tracker(type_of_tracker):
    if type_of_tracker == type_of_trackers[0]:
        tracker = cv2.legacy.TrackerBoosting_create()
    elif type_of_tracker == type_of_trackers[1]:
        tracker = cv2.legacy.TrackerMIL_create()
    elif type_of_tracker == type_of_trackers[2]:
        tracker = cv2.legacy.TrackerKCF_create()
    elif type_of_tracker == type_of_trackers[3]:
        tracker = cv2.legacy.TrackerTLD_create()
    elif type_of_tracker == type_of_trackers[4]:
        tracker = cv2.legacy.TrackerMedianFlow_create()
    elif type_of_tracker == type_of_trackers[5]:
        tracker = cv2.legacy.TrackerGOTURN_create()
    elif type_of_tracker == type_of_trackers[6]:
        tracker = cv2.legacy.TrackerMOSSE_create()
    elif type_of_tracker == type_of_trackers[7]:
        tracker = cv2.legacy.TrackerCSRT_create()
    else:
        tracker = None
        print('The name of the tracker is incorrect')
        print('Here are the possible trackers:')
        for track_type in type_of_trackers:
              print(track_type)
    return tracker

In [ ]:
def deep_learning():
    print("Deep Face threading is started!")
    models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
    model_name = "Facenet"
    recognizor = DeepFace.build_model(model_name)
    threshold = 0.85
    while cf.RUN:
        keys = cf.online_streams
        if len(keys)>0:
            for key in keys:
                outputs = []
                faces = face_detection(cf.inputs[key])
                for face in faces: 
                    face, pos = face[0], face[1]
                    if face.shape[0]*face.shape[1]>0:
                        min_distance, user_matched = user_verify(face, threshold = threshold)
                        t6 = time.time()
                        user, rect_color = [user_matched['name'], (0, 255, 0)] if user_matched is not None else ["Unknown", (255, 0, 0)]
                        if cf.use_tracking:
                            box = [pos[0], pos[1], pos[2]-pos[0], pos[3]-pos[1]]
                            tracker = tracker = generate_tracker(cf.desired_tracker)
                            tracker.init(cf.inputs[key], box)
                            outputs.append([[pos, user, min_distance, rect_color], tracker])
                        else:
                            outputs.append([[pos, user, min_distance, rect_color], None])
                inference_time = time.time()
                cf.outputs[key] = [outputs, inference_time]
        else:
            time.sleep(0.01)
    print("deep_learning threading is stopped!")
    
def video_stream_process(thread_name, link_index):
    print("Opening video stream ", thread_name)
    cap = cv2.VideoCapture(cf.video_links[link_index])
    while cf.RUN:
        ret, frame = cap.read()
        if not ret:
            break
        cf.inputs[cf.video_links[link_index]] = frame
        if not cf.video_links[link_index] in  cf.online_streams:
            cf.online_streams.append(cf.video_links[link_index])
        time.sleep(0.025) 
    print("Video stream ", thread_name, "is stopped!")
    

In [ ]:
import config as cf

import threading
import cv2
import time

link_index = 0
cf.desired_tracker = type_of_trackers[2]
cf.RUN = True
cf.video_links = ["1.mp4", "2.mp4", "3.mp4"]
cf.inputs = {}
cf.outputs = {}
cf.online_streams = []
cf.use_tracking = True

deep_learning_thread = threading.Thread(target=deep_learning)
deep_learning_thread.start()
time.sleep(0.2)
video_stream_process_thread_1 = threading.Thread(target=video_stream_process, args=("link 1", 0))
video_stream_process_thread_1.start()
time.sleep(0.2)
video_stream_process_thread_2 = threading.Thread(target=video_stream_process, args=("link 2", 1))
video_stream_process_thread_2.start()

In [ ]:
time.sleep(1)
font = cv2.FONT_HERSHEY_SIMPLEX
# fontScale
fontScale = 0.7
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2
link = cf.video_links[link_index]
while True:
    if link in cf.online_streams:
        img = cf.inputs[link]
        for output in cf.outputs[link][0]:
            pos, user, min_distance, rect_color = output[0]
            tracker = output[1]
            if cf.use_tracking:
                s, box = tracker.update(img)
            pos = [int(box[0]), int(box[1]), int(box[0])+int(box[2]), int(box[1])+int(box[3])]
            cv2.rectangle(img, (pos[0], pos[1]), (pos[2], pos[3]), rect_color, 2)
            cv2.putText(img, user +"("+str(round(min_distance, 2))+")", (pos[0], pos[1]-10), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(img, "Online Streams: "+str(cf.online_streams), (30, 30), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(img, "Current Stream: "+link, (30, 60), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow("frame", img)
        k = cv2.waitKey(33)
        if k == ord('q'):
            cf.RUN =False
            cv2.destroyAllWindows()
            break
        if k == 32:
            link_index+=1
            link_index = link_index%len(cf.online_streams)
            link = cf.online_streams[link_index]